In [3]:
import cv2
import time
import requests
import re
import numpy as np
from typing import Optional
from ultralytics import YOLO
from sort.sort import Sort
from util import get_car, reader

BACKEND_URL = "http://visaonestbackend-env.eba-25r2yw9h.us-east-2.elasticbeanstalk.com/cars/detect"

CHAR_TO_DIGIT = {'O': '0', 'I': '1', 'J': '3', 'A': '4', 'G': '6', 'S': '5'}
DIGIT_TO_CHAR = {v: k for k, v in CHAR_TO_DIGIT.items()}

PATTERN_OLD      = re.compile(r'^[A-Z]{3}\d{4}$')
PATTERN_MERCOSUL = re.compile(r'^[A-Z]{3}\d[A-Z]\d{2}$')

def normalize_plate(text: str) -> Optional[str]:
    cleaned = re.sub(r'[^A-Za-z0-9]', '', text.upper().replace(' ', ''))
    if cleaned.startswith("BR"):
        cleaned = cleaned[2:]
    if not cleaned:
        return None

    chars = list(cleaned)
    for i in range(len(chars)):
        if i in (0, 1, 4, 5, 6):
            chars[i] = DIGIT_TO_CHAR.get(chars[i], chars[i])
        elif i in (2, 3):
            chars[i] = CHAR_TO_DIGIT.get(chars[i], chars[i])
    plate = ''.join(chars)
    if len(plate) != 7:
        return None
    if PATTERN_OLD.fullmatch(plate) or PATTERN_MERCOSUL.fullmatch(plate):
        return plate
    return None

def sanitize_plate_for_sending(plate: str) -> str:
    return re.sub(r'[^A-Za-z0-9]', '', plate.upper().strip())

def extract_plate_text(thresh_img):
    for _, text, score in reader.readtext(thresh_img, detail=1):
        plate = normalize_plate(text)
        print(f"    OCR raw='{text}' -> normalizado='{plate}' (score={score:.2f})")
        if plate:
            return plate
    return None

def send_detection(track_id: int, plate: str):
    cleaned_plate = sanitize_plate_for_sending(plate)
    payload = {"trackId": track_id, "plate": cleaned_plate}
    print(f"\nEnviando payload: {payload}")
    try:
        r = requests.post(BACKEND_URL, json=payload, timeout=5)
        r.raise_for_status()
        print(f"{r.status_code}: Veículo cadastrado com sucesso.")
    except requests.RequestException as e:
        print(f"Falha HTTP: {e}")

def preprocess_crop(crop):
    gray = cv2.cvtColor(crop, cv2.COLOR_BGR2GRAY)
    gray = cv2.resize(gray, None, fx=2, fy=2, interpolation=cv2.INTER_LINEAR)
    return cv2.adaptiveThreshold(
        gray, 255,
        cv2.ADAPTIVE_THRESH_MEAN_C,
        cv2.THRESH_BINARY_INV, 35, 15)

def main():
    veh_model = YOLO("yolov8n.pt")
    lp_model  = YOLO("/home/messyas/ml/jetson/placas-model/models/runs/yolov8n_lp/weights/best.pt")
    tracker   = Sort()

    cap = cv2.VideoCapture(0, cv2.CAP_V4L2)
    if not cap.isOpened():
        raise RuntimeError("Não foi possível abrir a camera")

    DETECTION_INTERVAL = 5.0  # intervalo em segundos
    last_detection = time.time() - DETECTION_INTERVAL

    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                print("Falha ao capturar frame.")
                break

            # Mostra o feed em tempo real
            cv2.imshow("Webcam", frame)
            if cv2.waitKey(1) & 0xFF == ord("q"):
                break

            # A cada intervalo, congela este frame para deteccao
            now = time.time()
            if now - last_detection >= DETECTION_INTERVAL:
                last_detection = now
                print(f"\n=== Detecção @ {time.strftime('%H:%M:%S')} ===")

                # 1) Detecta veiculos
                results = veh_model(frame)[0]
                veh_boxes = [
                    b for b in results.boxes.data.tolist()
                    if int(b[5]) in [2, 3, 5, 7]
                ]

                # 2) Atualiza tracker apenas se houver deteccoes
                if veh_boxes:
                    dets = np.array([
                        [x1, y1, x2, y2, score]
                        for x1, y1, x2, y2, score, _ in veh_boxes
                    ], dtype=float)
                    tracks = tracker.update(dets)
                else:
                    tracks = []

                # 3) Detecta placas e envia ao backend
                lp_results = lp_model(frame)[0]
                for box in lp_results.boxes.data.tolist():
                    x1, y1, x2, y2, score, _ = box
                    *_, track_id = get_car(box, tracks)
                    if track_id == -1:
                        continue

                    crop = frame[int(y1):int(y2), int(x1):int(x2)]
                    plate = extract_plate_text(preprocess_crop(crop))
                    if plate:
                        print(f"Placa final: {plate}")
                        send_detection(int(track_id), plate)
                    else:
                        print("Nenhuma placa valida detectada.")
    except KeyboardInterrupt:
        print("Encerrado pelo usuario.")
    finally:
        cap.release()
        cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


=== Detecção @ 20:10:58 ===

0: 480x640 1 tie, 39.7ms
Speed: 3.0ms preprocess, 39.7ms inference, 284.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 6.1ms
Speed: 0.6ms preprocess, 6.1ms inference, 0.3ms postprocess per image at shape (1, 3, 480, 640)

=== Detecção @ 20:11:03 ===

0: 480x640 (no detections), 5.2ms
Speed: 1.0ms preprocess, 5.2ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 3.9ms
Speed: 0.9ms preprocess, 3.9ms inference, 0.3ms postprocess per image at shape (1, 3, 480, 640)

=== Detecção @ 20:11:08 ===

0: 480x640 (no detections), 4.4ms
Speed: 0.8ms preprocess, 4.4ms inference, 0.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 4.0ms
Speed: 0.8ms preprocess, 4.0ms inference, 0.3ms postprocess per image at shape (1, 3, 480, 640)

=== Detecção @ 20:11:13 ===

0: 480x640 (no detections), 4.4ms
Speed: 0.9ms preprocess, 4.4ms inference, 0.3ms postprocess per image